### Create a file that contains a list of file paths which store the graph files

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
%reload_ext autoreload
%autoreload 2
import scipy.stats as st
import os
from scipy.optimize import curve_fit
import igraph as ig
import csv
import networkx as nx
from networkx.algorithms import bipartite

In [ ]:
adj = []
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site1_WesternSahara1.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site2_WesternSahara2.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site3_Fuerteventura1.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site4_Fuerteventura2.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site5_GranCanaria1.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site6_GranCanaria2.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site7_TenerifeSouth1.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site8_TenerifeSouth2.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site9_TenerifeTeno1.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site10_TenerifeTeno2.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site11_Gomera1.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site12_Gomera2.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site13_Hierro1.csv"))
adj.append(pd.read_csv("/Users/anandnair/Documents/My Documents/Lifelong Learning/Santa Fe Institute/Multi-layer Network Project/Datasets/Plant Pollinator/Site14_Hierro2.csv"))

In [2]:
base_dir = "../../Datasets/Plant Pollinator/"

In [3]:
#adj = [pd.read_csv(base_dir+f) for f in os.listdir(base_dir) \
#       if f.startswith('Site')]
file_list = sorted([f.split('.')[0] for f in os.listdir(base_dir) \
                    if f.startswith('Site') and f.endswith('.csv') and\
                    'Links' not in f and 'Nodes' not in f])
print file_list
adj = [pd.read_csv(base_dir+f+'.csv') for f in file_list]
#adj[0]

['Site10_TenerifeTeno2', 'Site11_Gomera1', 'Site12_Gomera2', 'Site13_Hierro1', 'Site14_Hierro2', 'Site1_WesternSahara1', 'Site2_WesternSahara2', 'Site3_Fuerteventura1', 'Site4_Fuerteventura2', 'Site5_GranCanaria1', 'Site6_GranCanaria2', 'Site7_TenerifeSouth1', 'Site8_TenerifeSouth2', 'Site9_TenerifeTeno1']


In [4]:
cols = []
for i in range(len(adj)):
    cols.append(adj[i].columns.tolist())
cols = set([item for sublist in cols for item in sublist])

rows = []
for i in range(len(adj)):
    rows.append(adj[i].index.tolist())
rows = set([item for sublist in rows for item in sublist])

In [5]:
#pad out rows
for i in range(len(adj)):
    to_app = list(rows - set(adj[i].index.tolist()))
    for j in range(len(to_app)):
        name = to_app[j]
        series = pd.Series(list(np.zeros(len(adj[i].columns.tolist()))), index = adj[i].columns.tolist())
        series.name = name
        adj[i] = adj[i].append(series)
            
#rows looks good, will have to sort later
#now columns
#looks good!

for i in range(len(adj)):
    to_app = list(cols - set(adj[i].columns.tolist()))
    for j in range(len(to_app)):
        adj[i][to_app[j]] = 0

In [6]:
#all sorted, so hopefully now the node indexing will be consistent
for i in range(len(adj)):
    adj[i] = adj[i].sort_index()
    adj[i] = adj[i].sort_index(axis = 1)

In [7]:
adj[0].columns == adj[3].columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True], dtype=bool)

In [8]:
for item in adj:
    print item.shape

(249, 39)
(249, 39)
(249, 39)
(249, 39)
(249, 39)
(249, 39)
(249, 39)
(249, 39)
(249, 39)
(249, 39)
(249, 39)
(249, 39)
(249, 39)
(249, 39)


In [9]:
type(adj[0])

pandas.core.frame.DataFrame

In [10]:
node_list = adj[0].index.tolist() + adj[0].columns.tolist()
len(node_list)

288

In [30]:
node_list[0]

'Agromyzidae sp. 1'

In [11]:
for index in range(len(adj)):
    item = adj[index]
    print file_list[index]
    item.index = range(len(item.index))
    item.columns = range(len(item.index), len(item.index)+len(item.columns))
    print item
    item.to_csv('./%s_adjacency.csv'%file_list[index])

Site10_TenerifeTeno2
      249  250  251  252  253  254  255  256   257  258 ...    278  279  280  \
0     0.0    0    0  0.0  0.0  0.0    0  0.0   0.0  0.0 ...    0.0  0.0    0   
1     0.0    0    0  0.0  0.0  0.0    0  0.0   0.0  0.0 ...    0.0  1.0    0   
2     0.0    0    0  0.0  0.0  0.0    0  0.0   0.0  0.0 ...    0.0  0.0    0   
3     0.0    0    0  0.0  0.0  0.0    0  0.0   0.0  0.0 ...    0.0  0.0    0   
4     0.0    0    0  0.0  0.0  0.0    0  0.0   0.0  0.0 ...    0.0  0.0    0   
5     0.0    0    0  0.0  0.0  0.0    0  0.0   0.0  0.0 ...    0.0  0.0    0   
6     0.0    0    0  0.0  0.0  0.0    0  0.0   0.0  0.0 ...    0.0  0.0    0   
7     0.0    0    0  0.0  0.0  0.0    0  0.0   0.0  0.0 ...    0.0  0.0    0   
8     0.0    0    0  0.0  0.0  0.0    0  0.0   0.0  0.0 ...    0.0  0.0    0   
9     0.0    0    0  0.0  0.0  0.0    0  0.0   0.0  0.0 ...    0.0  0.0    0   
10    0.0    0    0  0.0  4.0  0.0    0  0.0  19.0  0.0 ...   19.0  3.0    0   
11    0.0    0    0

     249  250  251  252  253  254  255  256   257  258 ...   278  279  280  \
0      0    0    0  0.0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
1      0    0    0  0.0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
2      0    0    0  0.0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
3      0    0    0  0.0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
4      0    0    0  0.0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
5      0    0    0  0.0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
6      0    0    0  0.0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
7      0    0    0  0.0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
8      0    0    0  0.0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
9      0    0    0  0.0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
10     0    0    0  0.0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
11     0    0    0  0.0  0.0    0    0  0.0   0.0  0.0 ...     0

     249  250  251  252  253  254  255  256   257  258 ...   278  279  280  \
0      0    0    0    0    0    0    0  0.0   0.0    0 ...     0    0    0   
1      0    0    0    0    0    0    0  0.0   0.0    0 ...     0    0    0   
2      0    0    0    0    0    0    0  0.0   0.0    0 ...     0    0    0   
3      0    0    0    0    0    0    0  0.0   0.0    0 ...     0    0    0   
4      0    0    0    0    0    0    0  0.0   0.0    0 ...     0    0    0   
5      0    0    0    0    0    0    0  0.0   0.0    0 ...     0    0    0   
6      0    0    0    0    0    0    0  0.0   0.0    0 ...     0    0    0   
7      0    0    0    0    0    0    0  0.0   0.0    0 ...     0    0    0   
8      0    0    0    0    0    0    0  0.0   0.0    0 ...     0    0    0   
9      0    0    0    0    0    0    0  0.0   0.0    0 ...     0    0    0   
10     0    0    0    0    0    0    0  0.0   8.0    0 ...     0    0    0   
11     0    0    0    0    0    0    0  0.0   0.0    0 ...     0

      249  250  251  252  253  254  255  256   257  258 ...   278  279  280  \
0     0.0    0    0    0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
1     0.0    0    0    0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
2     0.0    0    0    0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
3     0.0    0    0    0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
4     0.0    0    0    0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
5     0.0    0    0    0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
6     0.0    0    0    0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
7     0.0    0    0    0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
8     0.0    0    0    0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
9     0.0    0    0    0  0.0    0    0  0.0   0.0  0.0 ...     0  0.0    0   
10    0.0    0    0    0  0.0    0    0  0.0  21.0  0.0 ...     0  1.0    0   
11    0.0    0    0    0  0.0    0    0  0.0   0.0  

#### Create `edge lists` from `adj. matrices`

In [12]:
# This module is imported so that we can use listdir or other function
import os 

In [13]:
'''To append the file path and file name that needs to be included in the adjacency matrix list. 
Endswith just indicates which files to include and the one with "Dryad" should be excluded.
'''
adj_matrices_list = sorted([f for f in os.listdir("./") \
                            if f.endswith('adjacency.csv')])
adj_matrices_list

['Site10_TenerifeTeno2_adjacency.csv',
 'Site11_Gomera1_adjacency.csv',
 'Site12_Gomera2_adjacency.csv',
 'Site13_Hierro1_adjacency.csv',
 'Site14_Hierro2_adjacency.csv',
 'Site1_WesternSahara1_adjacency.csv',
 'Site2_WesternSahara2_adjacency.csv',
 'Site3_Fuerteventura1_adjacency.csv',
 'Site4_Fuerteventura2_adjacency.csv',
 'Site5_GranCanaria1_adjacency.csv',
 'Site6_GranCanaria2_adjacency.csv',
 'Site7_TenerifeSouth1_adjacency.csv',
 'Site8_TenerifeSouth2_adjacency.csv',
 'Site9_TenerifeTeno1_adjacency.csv']

In [14]:
# Importing modules that are needed to run multired.py
import networkx as nx
import pandas as pd
import numpy as np
import scipy as sp
import csv

In [16]:
'''
Reading the csv file. Since the first column is also the node name therefore "index_col=0" tells panda to 
make the first column as the row name
'''
df_matrices = [pd.read_csv(adj_mat, index_col=0) for adj_mat in adj_matrices_list]

In [21]:
df_matrices[0].iloc[10]

249     0.0
250     0.0
251     0.0
252     0.0
253     4.0
254     0.0
255     0.0
256     0.0
257    19.0
258     0.0
259     0.0
260     0.0
261     0.0
262     0.0
263     0.0
264     0.0
265     0.0
266     0.0
267     0.0
268     0.0
269     0.0
270     0.0
271     0.0
272     0.0
273     0.0
274     0.0
275     0.0
276     0.0
277     0.0
278    19.0
279     3.0
280     0.0
281     0.0
282     0.0
283     0.0
284     0.0
285     0.0
286     0.0
287     0.0
Name: 10, dtype: float64

In [22]:
# Iterating through each matrix
for index in range(len(df_matrices)):
    mat = df_matrices[index] 
# Building a sparse matrix for networkx because networkx requires it for bipartite
    A = sp.sparse.csc_matrix(mat)
    g = nx.algorithms.bipartite.matrix.from_biadjacency_matrix(A)
    #bi_dict = nx.get_node_attributes(g, 'bipartite')
    #nx.set_node_attributes(g, 'bipartite', {k: 'pollinator' if bi_dict[k] == 0 \
    #                                        else 'plant' for k in bi_dict})
    #labels_list = mat.index.tolist() + mat.columns.tolist()
    #nx.relabel_nodes(g, {k: labels_list[k] for k in list(g)}, False)
# The above code is commented because it is not needed since we don't need the labels.

# Saving the edgelist to a csv file.
#print adj_matrices_list[index].split('adjacency')[0]
    with open('./%sedge_list.csv'%adj_matrices_list[index].split('adjacency')[0], 'w') as edge_list_f:
        csv.writer(edge_list_f, delimiter=' ').writerows(g.edges())

In [23]:
# Creating a csv that will be used by multired

edge_list_file = [f for f in os.listdir('./') if f.endswith('edge_list.csv')]
edge_list_file

['Site10_TenerifeTeno2_edge_list.csv',
 'Site11_Gomera1_edge_list.csv',
 'Site12_Gomera2_edge_list.csv',
 'Site13_Hierro1_edge_list.csv',
 'Site14_Hierro2_edge_list.csv',
 'Site1_WesternSahara1_edge_list.csv',
 'Site2_WesternSahara2_edge_list.csv',
 'Site3_Fuerteventura1_edge_list.csv',
 'Site4_Fuerteventura2_edge_list.csv',
 'Site5_GranCanaria1_edge_list.csv',
 'Site6_GranCanaria2_edge_list.csv',
 'Site7_TenerifeSouth1_edge_list.csv',
 'Site8_TenerifeSouth2_edge_list.csv',
 'Site9_TenerifeTeno1_edge_list.csv']

In [24]:
# Creating a csv that will be used by multired (contd). Here we are actually writing to a file.
with open('edge_list_file.csv', 'w') as f:
    csv.writer(f).writerows([[item] for item in edge_list_file])

Use `multired.py` 

In [25]:
import multired as mr

In [26]:
# Instructions in github

multiplex_red = mr.multiplex_red('./edge_list_file.csv')

In [27]:
# Function to calculate Jensen Shannon Divergence

multiplex_red.compute_JSD_matrix()

In [28]:
df = pd.DataFrame(multiplex_red.JSD)
df.index = edge_list_file
df.columns = edge_list_file
df

,Site10_TenerifeTeno2_edge_list.csv,Site11_Gomera1_edge_list.csv,Site12_Gomera2_edge_list.csv,Site13_Hierro1_edge_list.csv,Site14_Hierro2_edge_list.csv,Site1_WesternSahara1_edge_list.csv,Site2_WesternSahara2_edge_list.csv,Site3_Fuerteventura1_edge_list.csv,Site4_Fuerteventura2_edge_list.csv,Site5_GranCanaria1_edge_list.csv,Site6_GranCanaria2_edge_list.csv,Site7_TenerifeSouth1_edge_list.csv,Site8_TenerifeSouth2_edge_list.csv,Site9_TenerifeTeno1_edge_list.csv
Site10_TenerifeTeno2_edge_list.csv,0.000000,0.629494,0.599531,0.669942,0.618471,0.753540,0.738258,0.722910,0.722895,0.634156,0.652517,0.592429,0.575913,0.467994
Site11_Gomera1_edge_list.csv,0.629494,0.000000,0.428215,0.628562,0.603390,0.732058,0.736707,0.689671,0.704461,0.646172,0.636108,0.602475,0.604300,0.576904
Site12_Gomera2_edge_list.csv,0.599531,0.428215,0.000000,0.618591,0.611295,0.709087,0.695632,0.638188,0.667893,0.595680,0.586437,0.596297,0.592402,0.584857
Site13_Hierro1_edge_list.csv,0.669942,0.628562,0.618591,0.000000,0.478713,0.770740,0.777551,0.740728,0.752962,0.684063,0.694961,0.666155,0.689586,0.640747
Site14_Hierro2_edge_list.csv,0.618471,0.603390,0.611295,0.478713,0.000000,0.755200,0.757888,0.728035,0.730552,0.677285,0.697399,0.641115,0.651078,0.601101
Site1_WesternSahara1_edge_list.csv,0.753540,0.732058,0.709087,0.770740,0.755200,0.000000,0.485229,0.649548,0.637274,0.712560,0.713110,0.742123,0.743600,0.733244
Site2_WesternSahara2_edge_list.csv,0.738258,0.736707,0.695632,0.777551,0.757888,0.485229,0.000000,0.636986,0.630967,0.698460,0.688033,0.734044,0.738256,0.724035
Site3_Fuerteventura1_edge_list.csv,0.722910,0.689671,0.638188,0.740728,0.728035,0.649548,0.636986,0.000000,0.441820,0.647549,0.629296,0.683889,0.688224,0.689662
Site4_Fuerteventura2_edge_list.csv,0.722895,0.704461,0.667893,0.752962,0.730552,0.637274,0.630967,0.441820,0.000000,0.656452,0.656258,0.691816,0.699007,0.694169
Site5_GranCanaria1_edge_list.csv,0.634156,0.646172,0.595680,0.684063,0.677285,0.712560,0.698460,0.647549,0.656452,0.000000,0.427719,0.612774,0.632340,0.608551


In [29]:
df.to_csv('./JSD_06292017.csv')